<div class="alert alert-block" style = "background-color: black">
    <p><b><font size="+4" color="orange">Other Issues in Regression</font></b></p>
    <p><b><font size="+2" color="white">Extending the Linear Model</font></b></p>
    </div>

---

* **Import Packages**

In [1]:
#Import Data Manipulation Packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
from sklearn.model_selection import train_test_split
import scipy
from scipy import linalg
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

* **Import Data Visualization packages**

In [2]:
import seaborn as sb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams.update({'font.size':12}) #sets global font size



Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


---
<div class="alert alert-block" style="background-color: black">
    <p><b><font size="+2" color="white">Introduction</font></b></p>
    </div>
    
---

The Linear regression model works quite well by providing interepretable results on real-world problems but these results are based on restrictive assumptions that are often against the norm in practice. 

Two of these most common assumptions state the following that the relationship between the predictors and response are:

* **Additive** - This implies that the association between a predictor variable and the response does not depend on the value of other predictors 

* **& Linear** - This implies that the change in the response caused by a unit-change in a predictor variable is constant regardless of the value of that predictor variable.

Sophisticated techniques exist to address these two challenges and would be discussed in a different lesson.

---
<div class="alert alert-block" style="background-color: black">
    <p><b><font size="+2" color="orange">1. Removing the Additive Assumption by adding an Interaction term</font></b></p>
    </div>
    
---

Going back to the Advertising-Sales problem, the linear models derived from the multiple linear regression model assumed that the effect of increasing one advertising medium (e.g TV) on sales does not affect the amount spent on the other type of media (e.g radio & newspaper). 

Considering the linear model used for the multiple linear regression:

$$
\begin{align}
sales = \beta_{0} + \beta_{1}TV + \beta_{2}radio + \beta_{3}newspaper + ∊ \\
\end{align}
$$

This model implies that the average increase in sales associated with one-unit increase in **TV** will always be $\beta_{1}$ regardless of the amount spent on **radio** or **newspaper**. This simple model may be incorrect. Suppose spending money on radio advertising actually increases the effectiveness of TV advertising, so that the coefficient term for the TV increases as the radio increases. For instance, if given a fixed budget of $200,000, spending half on radio and half on TV may increase sales more than allocating the entire amount to either TV or radio. In statistics, this is called the **interaction effect**

Interaction effect may be present in the advertising data, lets investigate this:
Consider a standard linear regression model with two variables, $X_{1}$ & $X_{2}$:

$$
Y = \beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + ∈
$$

Lets try and understand what this model is telling us. According to this model, one-unit increase in $X_{1}$ leads to an average increase in Y by $\beta_{1}$ units. The prescence of $X_{2}$ does not alter this statement regardless of the value of $X_{2}$. 

One way of sudying the inter-relatiionship between these predictor variables $X_{1} and X_{2}$ and the response variable $Y$ is by introducing an **interaction term**. This **interaction** term can be computed as the product of two predictor variables - $X_{1} and X_{2}$.

$$
Y = \beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \beta_{3}(X_{1} × X_{2}) + ∈
$$

Separating  this into $X_{1}$ and $X_{2}$ terms gives:

$$
Y = \beta_{0} + (\beta_{1}+\beta_{3}X_{2})X_{1} + \beta_{2}X_{2} + ∈
$$

So the new $\beta_{1}$ term is $\hat \beta_{1} = \beta_{1}+\beta_{3}X_{2}$ which is now also a function of $X_{2}$. Hence the relationship between $X_{1} and Y$ is no longer a constant. A change in the value of $X_{2}$ will change the association between $X_{1}$ and $Y$

**Let's now go back and reformulate the advertising problem with an interaction term between TV and radio advert budgets to predict sales** 

---
## **Example - Multiple Linear Regression with interaction term**

$$
\begin{align}
sales = \beta_{0} + \beta_{1}TV + \beta_{2}radio + \beta_{3}(radio × TV) + ∊ \\
= \beta_{0} + (\beta_{1} + \beta_{3}radio)×TV + \beta_{2}radio + ∊ 
\end{align}
$$

$\beta_{3}$ can be interpreted as the increase in effectiveness of TV advertising associated with a unit increase in radio advertising (or vice-versa)

In [3]:
Advert = pd.read_csv("../Data/ISLP_Data/Advertising.csv")
Advert.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/ISLP_Data/Advertising.csv'

* State your independent,X and dependent variables,Y

In [ ]:
X = pd.DataFrame({'TV':Advert['TV'],'Radio':Advert['radio'],'TV & Radio':Advert['TV']*Advert['radio']})
Y = Advert['sales']

* Use the stats model to perform the linear regression

In [ ]:
X = sm.add_constant(X)
MLR_interaction_TVRad = sm.OLS(Y,X).fit() #Perform the least squares fit
predicted_MLR_interaction_TVRad = MLR_interaction_TVRad.predict(X) #retrieve your predicted Y values
MLR_interaction_TVRad.summary() #Display your results

* Let's compare this with the model with no interaction term

In [ ]:
X_noint = pd.DataFrame({'TV':Advert['TV'],'Radio':Advert['radio']})
X_noint = sm.add_constant(X_noint)
MLR_no_interaction_TVRad = sm.OLS(Y,X_noint).fit() #Perform the least squares fit
predicted_MLR_no_interaction_TVRad = MLR_no_interaction_TVRad.predict(X_noint) #retrieve your predicted Y values
MLR_no_interaction_TVRad.summary() #Display your results

### **Analysis**

The results presented above clearly show that the model which includes the interaction term is better and superior to the model with no interatcion term. 
* The p-value for the interaction term is extremely low, indicatiing that there's a strong evidence for the alternative hypothesis,H_{a}: $\beta_{3}$. This proves to show that the true relationship is not additive.
* The $R^{2}$ for the model is 96.8% compared to the model with no interaction term.
* The effect of this interaction term is clearly evident in the increment in the $R_{2}$ value. A difference of (96.8 - 89.7) = 7.1%

So therefore, if we increase TV advertising by 1000, sales will increase by $(\hat \beta_{1} + \hat \beta_{3} × radio)×1000 = 19 + 1.1 × radio$ where 19 & 1.1 are the coefficients multiplied by 1000.

Likewise, if we increase radio advertising by 1000, sales will increase by: We rewrite the model by factoring out the radio units.

$$
\begin{align}
sales = \beta_{0} + \beta_{1}TV + \beta_{2}radio + \beta_{3}(radio × TV) + ∊ \\
= \beta_{0} + (\beta_{1} + \beta_{3}radio)TV + (\beta_{2} + \beta_{3}TV)radio + ∊ 
\end{align}
$$

This gives: $(\beta_{2} + \beta_{3}TV) × 1000 = 29 + 1.1$ × TV units.

* The p-values associated with the 3 predictor variables -TV,radio and the interaction term are all statistically significant (very small). Hence, these 3 variables should be included in the model.

But there may be the case where the interaction term may have a very small p-value but the associated main effects (i.e TV and radio) do not.

>The guiding principle in this case is that *if we include an interaction in a model, we should also include the main effects, even if the p-values associated with their coefficients are not significant*
**If the interaction between $X_{1}$ and $X_{2}$ seems important, then we should include both $X_{1}$ and $X_{2}$ in the model even if their coefficient estimates have large p-values**

* The rationale for this guiding principle is that if $X_{1} × X_{2}$ is related to the response, then whether or not the coefficients of $X_{1} or X_{2}$ are exactly zero is of no importance.

* Also, $X_{1} × X_{2}$ is typically correlated with $X_{1} and X_{2}$ and so leaving them out tends to alter the significance of the interaction. 

---
<div class="alert alert-block" style="background-color: black">
    <p><b><font size="+2" color="orange">1.1 Interaction between Quantitative &  Qualitative variables</font></b></p>
    </div>
    
---

The above example illustrated the interaction between two quantitative variables - TV and Radio. But the concept of interaction applies just as well to qualitative variables and the interaction between a qualitative and quantitative variable has an interesting interpretation. This will be illustrated using the **Credit** dataset 

## **Illustration between Quantitative & Qualitative Variables with NO Interaction**
---

Considering the Credit dataset, suppose we want to predict **balance** using the **income**(quantitative) and **student**(qualitative) variables in the abscence of an interaction term, the formulation of the model takes the form: 

* Create the dummy variable for student, $x_{i}$

$$
x_{i} =
\begin{cases}
1 & \mathit {\text{if ith person is student}}\\
0 & \mathit {\text{if ith person is not a student}}
\end{cases}
$$

This leads to the formulation of the linear regression model

$$
balance_{i} ≈ \beta_{0} + \beta_{1}income_{i} + \beta_{2}x_{i} + ∈_{i} = \beta_{1}Income_{i} + 
\begin{cases}
\beta_{0} + \beta_{2} + ∈_{i} & \mathit{\text{if the ith person is a student}}\\
\beta_{0} + ∈_{i} & \mathit{\text{if the ith person is not a student}}
\end{cases}
$$

N.B: The final derived formulation for the linear regression above is the famous equation of a straight line $y = mx + c$

In [ ]:
Credit = pd.read_csv('../Data/ISLP_DATA/Credit.csv')
Credit.head()

* Create dummy variables for the student predictor

In [ ]:
Student_Status= Credit.Student.str.get_dummies('No')
Credit['Is_student_10'] = Student_Status
Credit.head()

**Perform Regression directly from the formula created above**

In [ ]:
MLR_Formula_Model_noint = sm.OLS.from_formula('Balance ~ Income + Is_student_10', Credit).fit()
MLR_Formula_Model_noint.summary()

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(Credit.Income,Credit.Balance,marker='o',color='black')
plt.plot(Credit.Income[Credit['Is_student_10']==1],MLR_Formula_Model_noint.fittedvalues[Credit['Is_student_10']==1],label='Student')
plt.plot(Credit.Income[Credit['Is_student_10']==0],MLR_Formula_Model_noint.fittedvalues[Credit['Is_student_10']==0],label='Non-Student')
plt.xlabel('Income')
plt.ylabel('Balance');
plt.legend();

## **Analysis**

From the plot above, we could observe two parallel regression lines fitted to the dataset. One for the students and one for non-students with intercepts $\beta_{0} + \beta_{2}$ and $\beta_{0} respectively$. 
The parallelization of the regression lines means that the average increase in credit card balance caused by a unit increase in income does not depend on whether the individual is a student or not. This represents a potentially serious limitation of the model since in actual fact, a change in income may have a very different effect on the credit card balance of a student versus a non-student.

## **Illustration between Quantitative & Qualitative Variables WITH Interaction**
---

The limitation observed above can be addressed by adding an interaction variable, created by multiplying income with the dummy variable for student. 

Remember the dummy variable is:
$$
x_{i} =
\begin{cases}
1 & \mathit {\text{if ith person is student}}\\
0 & \mathit {\text{if ith person is not a student}}
\end{cases}
$$

The model is then formulated thus:

$$
balance_{i} = \beta_{0} + \beta_{1}income_{i} + \beta_{2}x_{i} + \beta_{3}(income_{i}× x_{i}) + ∈ \\
= \beta_{0} + \beta_{1}income_{i} +
\begin{cases}
\beta_{2} + \beta_{3}× income_{i} & \mathit {\text{if student}}\\
0 & \mathit {\text{if not student}}
\end{cases}
$$ 

Separating out the terms into the two predictor variables - income and student dummy variable:

$$
balance_{i} = 
\begin{cases}
(\beta_{0} + \beta_{2}) + (\beta_{1} + \beta_{3}) × income_{i} & \mathit {\text{if student}}\\
\beta_{0} + \beta{1} × income_{i} & \mathit {\text{if not student}}
\end{cases}
$$

* **Perform the Regression based on the formula stated above**

In [ ]:
MLR_FormulaModel_w_int = sm.OLS.from_formula('Balance ~ Income + Is_student_10 + (Income * Is_student_10)',Credit).fit()
MLR_FormulaModel_w_int.summary()

In [ ]:
label=['student','non-student']
plt.figure(figsize=(10,8))
plt.scatter(Credit.Income,Credit.Balance,marker='o',color='black')
plt.plot(Credit.Income[Credit['Is_student_10']==1],MLR_FormulaModel_w_int.fittedvalues[Credit['Is_student_10']==1],label=label[0])
plt.plot(Credit.Income[Credit['Is_student_10']==0],MLR_FormulaModel_w_int.fittedvalues[Credit['Is_student_10']==0],label=label[1])
plt.title('Income vs Balance Relationship for Students & Non-Students with Interaction')
plt.xlabel('Income')
plt.ylabel('Balance');
plt.legend()
sb.lmplot(x="Income",y="Balance",hue="Student",data=Credit);

## **Analysis**

The plot above shows two regression lines again for the students and non-students. There is interaction between income and student

* The regression lines have different intercepts $\beta_{0} + \beta_{2} = 200.6 + 6.2 = 206.8$ versus $\beta_{0} = 200.6$ 
* and different slopes $\beta_{1} + \beta_{3} = 476.7 + (-1.99) = 474.7$ versus $\beta_{1} = 476.7$

This observation proves to show that changes in income could affect the credit card balances of students and non-students differently.
The slope for students is lower than the slope for non-students suggesting that increases in income are associated with smaller increases in credit card balance among students as compared to non-students.

---
<div class="alert alert-block" style="background-color: black">
    <p><b><font size="+2" color="orange">2. Non-Linearity via Polynomial Regression</font></b></p>
    </div>
    
---

As mentioned previously, that the linear regression model assumes a linear relationship between the response and the predictor variables. But in actual fact, there may be cases whee the underlying relationship between the response and the predictors may be non-linear.

In this section, we will extend the linear model to account for non-linear relationships using **polynomial regression** using the **Auto** dataset. We try to investigate the relationship between the horsepower of the vehicle against the miles per gallons used by that vehicle.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
Auto = pd.read_csv('../../../Data/ISLP_DATA/Auto.csv',na_values='?')
Auto = Auto.dropna(0,'any')
Auto.head()

* Get your Independent and dependedent variables 

In [ ]:
X_1 = pd.DataFrame({'Horsepower':Auto['horsepower']})
Y_1 = Auto['mpg']

* Make a scatter plot to explore the relationship between the two variables

In [ ]:
ax = Auto.plot.scatter('horsepower','mpg',marker='o',color='black',figsize=(7,4))
ax.set_xlabel('Horsepower')
ax.set_ylabel('Miles per gallon')
ax.set_title('Relationship btw Horsepower & Mpg');

There is a profund relationship between horsepower and miles per gallon but it is obvious that this relationship is infact non-linear. Rather than a constant declining slope between the two variables, we have a declining curve. We can account for this in our model.

A simple approach for incorporating non-linear associations in a linear modelis to include **transformed versions** of the predictor variables. The shape of the declining curve above seems to be quadratic in shape, hence our linear model can be extended to a quadratic form.

$$
mpg = \beta_{0} + \beta_{1} × horsepower + \beta_{2} × horsepower^{2} + ∈
$$

What this model is trying to achieve is to predict mpg using a non-linear function of horsepower. But it is still a linear model. It is a multiple linear regression model with predictor variable $X_{1} = horsepower$ and $X_{2} = horsepower^{2}$

* **Perform the Regression based on the relationship frmulated above**

In [ ]:
horsepower2 = np.power(Auto.horsepower,2)
MLR_NonLinearModel = sm.OLS.from_formula('mpg ~ horsepower + horsepower2 ',Auto).fit()
ypred_deg2 = MLR_NonLinearModel.predict()
print(summarize(MLR_NonLinearModel),"\n\n","Rsquared =",MLR_NonLinearModel.rsquared)

* **Perform the Regression to the 6th degree**

We use a list comprehension to populate the columns of the polynomials of higher degrees till we get to the 5th degree. This makes our code less cumbersome.

In [ ]:
MLR_PolyModel = sm.OLS.from_formula('mpg ~' + '+' .join(['np.power(horsepower,' + str(i) + ')' for i in range(1,7)]),Auto).fit()
ypred_deg6 = MLR_PolyModel.predict()
print(summarize(MLR_PolyModel),"\n\n","Rsquared =",MLR_PolyModel.rsquared)

* **Fit a linear model to the dataset**

In [ ]:
MLR_LinearModel = sm.OLS.from_formula('mpg ~ horsepower',Auto).fit()
ypred_deg1 = MLR_LinearModel.predict()
print(summarize(MLR_LinearModel),"\n","Rsquared =",MLR_LinearModel.rsquared)

* **Plot the 3 Regression Models**

In [ ]:
plt.figure(figsize=(11,7))
plt.scatter(Auto.horsepower,Auto.mpg,marker='o',color="silver")
plt.scatter(Auto['horsepower'],ypred_deg1,label='linear')
plt.scatter(Auto['horsepower'],ypred_deg2,label='2nd degree')
plt.scatter(Auto['horsepower'],ypred_deg5,label='6th degree')
plt.legend()
plt.xlabel('Horsepower')
plt.ylabel('Miles per gallon')
plt.title('Exploring Non-Linear relationship btw Horsepower & Mpg');

### **Analysis**

Evaluating the three Models using the R-squared value, this measurement tells us how much of the total variance of the response - **miles per gallons** is explained by the model. The higher the R-squared, the better the model explains the data. It is clearly observed that as the number of features or predictor variables is increased, the R-squared value increases. 

$$
\begin{align}
\begin{array}{c c c}
 & R^{2} \text{Values} & \\ 
Linear & \text{2nd degree} & \text{6th degree}\\
0.606  & 0.688             & 0.696
\end{array} 
\end{align}
$$

The 6th degree polynomial displays a fit includes all polynomials up to the 6th degree with a minimal increase in the R-squared value compared to the 2nd degree poynomial. This questions the need for a higher-order polynomial to model the data. It is worth mentioning that the R-squared value will increase as the number of predictors are increased regardless of the importance or contribution each predictor does to the overall model.